In [32]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from IPython.display import display
from functools import reduce

In [33]:
def merge_df(dataframes):
    """
    Merges multiple pandas DataFrames on their index.

    Parameters:
    dataframes (list of pandas.DataFrame): The list of DataFrames to merge.
    how (str): Type of merge to perform:
        - 'left': use only keys from left frame (SQL: left outer join)
        - 'right': use only keys from right frame (SQL: right outer join)
        - 'outer': use union of keys from both frames (SQL: full outer join)
        - 'inner': use intersection of keys from both frames (SQL: inner join)
    remove_duplicates (bool): Whether to remove duplicated rows after merging.

    Returns:
    pandas.DataFrame: The merged DataFrame.
    """
    # Start with the first DataFrame in the list
    merged_df = dataframes[0]

    # Iteratively merge each DataFrame in the list
    for df in dataframes[1:]:
        merged_df = merged_df.join(df, how='left')

    return merged_df


In [34]:
def concat_df(dataframes, axis='index'):
    # Identify common columns by intersecting all DataFrame columns
    common_columns = reduce(lambda x, y: x.intersection(y.columns), dataframes, dataframes[0].columns)

    # Reindex all DataFrames to these common columns
    dfs_common = [df[common_columns] for df in dataframes]

    # Concatenate the reindexed DataFrames
    return pd.concat(dfs_common, axis=axis)

In [35]:
def read_parquet(filepath,index=None,loc=None):
    df = pd.read_parquet(filepath)
    if index:
        df.set_index(index, inplace=True)
        df.index.name = 'index'
        df.index = pd.to_datetime(df.index)
        # df['timestamp'] = pd.to_datetime(df.index).astype('datetime64[ns]')
    return df

In [36]:
def get_min_time_delta_in_minutes(df):
    data_sorted = df.sort_index()
    time_deltas = data_sorted.index.to_series().diff().dropna()
    time_deltas_in_minutes = time_deltas.dt.total_seconds() / 60
    min_time_delta = time_deltas_in_minutes.min()
    return f"{min_time_delta:.0f}T"

# Start

In [37]:
Y_train           = read_parquet('/home/andres/ml/data/A/train_targets.parquet',     'time',         'A')
X_train_estimated = read_parquet('/home/andres/ml/data/A/X_train_estimated.parquet', 'date_forecast','A')
X_train_observed  = read_parquet('/home/andres/ml/data/A/X_train_observed.parquet',  'date_forecast','A')
X_test_estimated  = read_parquet('/home/andres/ml/data/A/X_test_estimated.parquet',  'date_forecast','A')

In [38]:
X_train = concat_df([X_train_estimated, X_train_observed])
X_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 136245 entries, 2022-10-28 22:00:00 to 2022-10-21 01:00:00
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   absolute_humidity_2m:gm3        136245 non-null  float32
 1   air_density_2m:kgm3             136245 non-null  float32
 2   ceiling_height_agl:m            110079 non-null  float32
 3   clear_sky_energy_1h:J           136245 non-null  float32
 4   clear_sky_rad:W                 136245 non-null  float32
 5   cloud_base_agl:m                126085 non-null  float32
 6   dew_or_rime:idx                 136245 non-null  float32
 7   dew_point_2m:K                  136245 non-null  float32
 8   diffuse_rad:W                   136245 non-null  float32
 9   diffuse_rad_1h:J                136245 non-null  float32
 10  direct_rad:W                    136245 non-null  float32
 11  direct_rad_1h:J                 136245 non-n

In [39]:
def interpolate(data):

    interpolation_methods = {
        'absolute_humidity_2m:gm3': 'index',
        'air_density_2m:kgm3': 'index',
        'ceiling_height_agl:m': 'index',
        'clear_sky_energy_1h:J': 'cubic',
        'clear_sky_rad:W': 'cubic',
        'cloud_base_agl:m': 'pchip',
        'dew_or_rime:idx': 'nearest',
        'dew_point_2m:K': 'linear',
        'diffuse_rad:W': 'cubic',
        'diffuse_rad_1h:J': 'cubic',
        'direct_rad:W': 'cubic',
        'direct_rad_1h:J': 'cubic',
        'effective_cloud_cover:p': 'index',
        'elevation:m': 'pad',
        'fresh_snow_12h:cm': 'zero',
        'fresh_snow_1h:cm': 'zero',
        'fresh_snow_24h:cm': 'zero',
        'fresh_snow_3h:cm': 'zero',
        'fresh_snow_6h:cm': 'zero',
        'is_day:idx': 'pad',
        'is_in_shadow:idx': 'pad',
        'msl_pressure:hPa': 'time',
        'precip_5min:mm': 'index',
        'precip_type_5min:idx': 'nearest',
        'pressure_100m:hPa': 'index',
        'pressure_50m:hPa': 'index',
        'prob_rime:p': 'index',
        'rain_water:kgm2': 'index',
        'relative_humidity_1000hPa:p': 'index',
        'sfc_pressure:hPa': 'time',
        'snow_density:kgm3': 'zero',
        'snow_depth:cm': 'nearest',
        'snow_drift:idx': 'pad',
        'snow_melt_10min:mm': 'index',
        'snow_water:kgm2': 'index',
        'sun_azimuth:d': 'cubic',
        'sun_elevation:d': 'cubic',
        'super_cooled_liquid_water:kgm2': 'index',
        't_1000hPa:K': 'index',
        'total_cloud_cover:p': 'index',
        'visibility:m': 'index',
        'wind_speed_10m:ms': 'index',
        'wind_speed_u_10m:ms': 'index',
        'wind_speed_v_10m:ms': 'index',
        'wind_speed_w_1000hPa:ms': 'index',
        'pv_measurement':'index'
    }
    for column in data.columns:
        data[column].interpolate(method=interpolation_methods.get(column, 'linear'), inplace=True)
        if data[column].isna().any():
            print(column + ' has NaN, replacing NaN with 0.')
            data[column] = data[column].fillna(0)
    return data

In [40]:
freq = get_min_time_delta_in_minutes(X_train)
X_train = X_train.resample(freq).asfreq(fill_value=None)
train_data = merge_df([interpolate(X_train),interpolate(Y_train)])
interpolate(train_data)


snow_density:kgm3 has NaN, replacing NaN with 0.


/tmp/ipykernel_36654/658097509.py:52: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[column].interpolate(method=interpolation_methods.get(column, 'linear'), inplace=True)
/tmp/ipykernel_36654/658097509.py:52: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[column].interpolate(method=interpolation_methods.get(column, 'linear'), inplace=True)
/tmp/ipykernel_36654/658097509.py:52: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[column].interpolate(method=interpolation_methods.get(column, 'linear'), inplace=True)
/tmp/ipykernel_36654/658097509.py:52: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[co

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,pv_measurement
index,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,0.0,...,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,0.0
2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,0.0,...,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,0.0
2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,0.0,...,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,0.0
2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,0.0,...,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,0.0
2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,0.0,...,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 22:45:00,4.5,1.281,1173.900024,0.0,0.0,536.500000,0.0,272.299988,0.0,0.0,...,-11.731,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0,0.0
2023-04-30 23:00:00,4.5,1.281,1054.199951,0.0,0.0,542.400024,0.0,272.200012,0.0,0.0,...,-11.884,0.1,274.200012,100.000000,9923.200195,3.7,2.8,2.4,-0.0,0.0
2023-04-30 23:15:00,4.5,1.281,1435.800049,0.0,0.0,531.799988,0.0,272.299988,0.0,0.0,...,-11.928,0.1,274.200012,99.900002,11230.799805,3.7,2.7,2.5,-0.0,0.0


In [31]:

train_data['timestamp'] = pd.to_datetime(train_data.index)
print(train_data['timestamp'])
train_data['location'] = 'A'
train_data = TimeSeriesDataFrame.from_data_frame(
    train_data,
    id_column="location",
    timestamp_column="timestamp",
)
# train_data = train_data.to_regular_index(get_min_time_delta_in_minutes(data))
# (train_data_s, test_data_s) = train_data.train_test_split(prediction_length=test_data.shape[0])


index
2019-06-02 22:00:00   2019-06-02 22:00:00
2019-06-02 22:15:00   2019-06-02 22:15:00
2019-06-02 22:30:00   2019-06-02 22:30:00
2019-06-02 22:45:00   2019-06-02 22:45:00
2019-06-02 23:00:00   2019-06-02 23:00:00
                              ...        
2023-04-30 22:45:00   2023-04-30 22:45:00
2023-04-30 23:00:00   2023-04-30 23:00:00
2023-04-30 23:15:00   2023-04-30 23:15:00
2023-04-30 23:30:00   2023-04-30 23:30:00
2023-04-30 23:45:00   2023-04-30 23:45:00
Freq: 15T, Name: timestamp, Length: 137096, dtype: datetime64[us]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 train_data['timestamp'] = pd.to_datetime(train_data.index)                                  │
│    2 print(train_data['timestamp'])                                                              │
│    3 train_data['location'] = 'A'                                                                │
│ ❱  4 train_data = TimeSeriesDataFrame.from_data_frame(                                           │
│    5 │   train_data,                                                                             │
│    6 │   id_column="location",                                                                   │
│    7 │   timestamp_column="timestamp",                                                           │
│                                                                                                  │
│ /home/andres/miniforge3/lib/python3.10/site-packages/autogluon/timeseries/dataset/ts_dataframe.p │
│ y:423 in from_data_frame                                                                         │
│                                                                                                  │
│   420 │   │   │   A data frame in TimeSeriesDataFrame format.                                    │
│   421 │   │   """                                                                                │
│   422 │   │   return cls(                                                                        │
│ ❱ 423 │   │   │   cls._construct_pandas_frame_from_data_frame(df, id_column=id_column, timesta   │
│   424 │   │   )                                                                                  │
│   425 │                                                                                          │
│   426 │   def copy(self: TimeSeriesDataFrame, deep: bool = True) -> pd.DataFrame:  # noqa        │
│                                                                                                  │
│ /home/andres/miniforge3/lib/python3.10/site-packages/autogluon/timeseries/dataset/ts_dataframe.p │
│ y:344 in _construct_pandas_frame_from_data_frame                                                 │
│                                                                                                  │
│   341 │   │   if TIMESTAMP in df.columns:                                                        │
│   342 │   │   │   df[TIMESTAMP] = pd.to_datetime(df[TIMESTAMP])                                  │
│   343 │   │                                                                                      │
│ ❱ 344 │   │   cls._validate_data_frame(df)                                                       │
│   345 │   │   return df.set_index([ITEMID, TIMESTAMP])                                           │
│   346 │                                                                                          │
│   347 │   @classmethod                                                                           │
│                                                                                                  │
│ /home/andres/miniforge3/lib/python3.10/site-packages/autogluon/timeseries/dataset/ts_dataframe.p │
│ y:230 in _validate_data_frame                                                                    │
│                                                                                                  │
│   227 │   │   if df[TIMESTAMP].isnull().any():                                                   │
│   228 │   │   │   raise ValueError(f"`{TIMESTAMP}` column can not have nan")                     │
│   229 │   │   if not df[TIMESTAMP].dtype == "datetime64[ns]":                                    │
│ ❱ 230 │   │   │   raise ValueError(f"for {TIMESTAMP}, the o

In [ ]:
try:
    X_test_estimated_a = X_test_estimated_a.drop('date_calc', axis=1)
except:
    None
test_data = fill_missing_values(merge_df([X_test_estimated_a, Y_train_a]))

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=test_data.shape[0],
    path="autogluon",
    target="pv_measurement",
    eval_metric="MSE",
)

predictor.fit(
    train_data_s,
    presets="fast_training",
    time_limit=600,
)

In [ ]:
print(predictor.leaderboard(test_data_s, silent=True))

In [ ]:
predictor.evaluate(test_data_s)

In [ ]:
predictions = predictor.predict(test_data)
print(predictions.info())